### Please install the required Python modules/SDKs

In [15]:
! activate ai-azure-c1

import sys

sys.path.append("/opt/conda/envs/ai-azure-c1/lib/python3.8/site-packages")

# Azure Custom Vision - Object Detection

## Import utility functions and Python modules 

In [16]:
import requests
from urllib.parse import urlparse
from io import BytesIO
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

import os, time, uuid

In [17]:
def show_image_in_cell(img_url):
    response = requests.get(img_url)
    img = Image.open(BytesIO(response.content))
    plt.figure(figsize=(20,10))
    plt.imshow(img)
    plt.show()

In [18]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials


### Resources:
- Azure Custom Vision Endpoint
- Training Reource ID and Key
- Prediction Resource ID and Key

## Make sure you have the correct Training and Prediction Endpoints, Keys and Resource IDs separately

### Your Training Endpoint resource must be for both training and prediction

In [19]:
TRAINING_ENDPOINT = "https://vision244721.cognitiveservices.azure.com/"
training_key = "8b3070d0b1804ca3824f6eb7ff9642f0"
training_resource_id = "/subscriptions/e144b1f9-a885-4518-b6ed-b2f3026e2a17/resourceGroups/ODL-AIND-244721/providers/Microsoft.CognitiveServices/accounts/vision244721"

In [20]:
PREDICTION_ENDPOINT = "https://vision244721-prediction.cognitiveservices.azure.com/"
prediction_key = "49a5da8324da4f07a7d84a2fb4c26e70"
prediction_resource_id = "/subscriptions/e144b1f9-a885-4518-b6ed-b2f3026e2a17/resourceGroups/ODL-AIND-244721/providers/Microsoft.CognitiveServices/accounts/vision244721-Prediction"

## Instantiate and authenticate the training client with endpoint and key 

In [21]:
training_credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(TRAINING_ENDPOINT, training_credentials)

In [22]:
trainer.api_version

'3.4-preview'

## Instantiate and authenticate the prediction client with endpoint and key

In [23]:
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(PREDICTION_ENDPOINT, prediction_credentials)

In [24]:
predictor.api_version

'3.1'

## Creating Training Project First

In [25]:
# Find the object detection domain
obj_detection_domain = next(domain for domain in trainer.get_domains() if domain.type == "ObjectDetection" and domain.name == "General")

# Create a new project
print ("Your Object Detection Training project has been created. Please move on.")
project_name = uuid.uuid4()
project = trainer.create_project(project_name, domain_id=obj_detection_domain.id)

Your Object Detection Training project has been created. Please move on.


## Getting Project Details as collective information 

In [26]:
project.as_dict()

{'id': 'e045b25d-aa6a-4423-805e-44398a6b0eb1',
 'name': 'e2722582-9ea7-492d-8bd4-94dc9857e524',
 'description': '',
 'settings': {'domain_id': 'da2e3a8a-40a5-4171-82f4-58522f70fbc1',
  'classification_type': 'Multilabel',
  'target_export_platforms': [],
  'use_negative_set': True,
  'image_processing_settings': {'augmentation_methods': {'rotation': True,
    'scaling': True,
    'translation': True,
    'horizontal flip': True,
    'equalize': True,
    'solarize': True,
    'padtosquare': True}}},
 'created': '2023-12-04T00:23:43.160Z',
 'last_modified': '2023-12-04T00:23:43.160Z',
 'dr_mode_enabled': False,
 'status': 'Succeeded'}

In [27]:
project.status

'Succeeded'

## Adding Tags based on training requirements
  
- In the demo, we used images and tags for Bird and Flower. For this exercise, you can use the "bottle" tag or any other class/type of your own images.
- Please modify the code accordingly.

In [28]:
lighter_tag = trainer.create_tag(project.id, "lighter")

flower_tag = trainer.create_tag(project.id, "Flower")

# VERY IMPORTANT - PAUSE HERE
# Now, please go to the Custom Vision portal, upload and label your training images
## Please read the following instructions:

- Please visit the Custom Vision Portal (https://computervision.api) and upload all your training images manually. 
- After that, you can label the tag region for each object directly at the portal. 
- This way, you don't need to use any third-party website or service to generate tag regions in the form of bounding box coordinates. If you have training images with those coordinates, you can use the optional section in the demo Jupyter Notebook to upload your images with the tagged objects via code.

### Once you have uploaded and labeled all the training images at the portal, you can come back to this notebook and start the training process. 

## Start the Object Detection Training
- We will keep checking every 10 seconds during the training progress

In [29]:
iteration = trainer.train_project(project.id)
while (iteration.status != "Completed"):
    iteration = trainer.get_iteration(project.id, iteration.id)
    print ("Training status: " + iteration.status)
    print ("Waiting 10 seconds...")
    time.sleep(10)

Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 sec

## After training is complete, we will check model performance

In [30]:
iteration.as_dict()

{'id': '418da9a8-e6f8-4dfd-9c2a-f77ff430b6e3',
 'name': 'Iteration 1',
 'status': 'Completed',
 'created': '2023-12-04T00:23:43.163Z',
 'last_modified': '2023-12-04T00:32:33.426Z',
 'trained_at': '2023-12-04T00:32:33.423Z',
 'project_id': 'e045b25d-aa6a-4423-805e-44398a6b0eb1',
 'exportable': False,
 'domain_id': 'da2e3a8a-40a5-4171-82f4-58522f70fbc1',
 'training_type': 'Regular',
 'reserved_budget_in_hours': 0,
 'training_time_in_minutes': 2}

In [31]:
iteration_list = trainer.get_iterations(project.id)
for iteration_item in iteration_list:
    print(iteration_item)

{'additional_properties': {}, 'id': '375c2bb8-9681-4fc8-9a43-4bccf72e8e72', 'name': 'Iteration 6', 'status': 'Completed', 'created': datetime.datetime(2023, 12, 4, 1, 11, 7, 903000, tzinfo=<isodate.tzinfo.Utc object at 0x7f9a90de62e8>), 'last_modified': datetime.datetime(2023, 12, 4, 1, 24, 6, 681000, tzinfo=<isodate.tzinfo.Utc object at 0x7f9a90de62e8>), 'trained_at': datetime.datetime(2023, 12, 4, 1, 24, 6, 678000, tzinfo=<isodate.tzinfo.Utc object at 0x7f9a90de62e8>), 'project_id': 'e045b25d-aa6a-4423-805e-44398a6b0eb1', 'exportable': False, 'exportable_to': None, 'domain_id': 'da2e3a8a-40a5-4171-82f4-58522f70fbc1', 'classification_type': None, 'training_type': 'Regular', 'reserved_budget_in_hours': 0, 'training_time_in_minutes': 2, 'publish_name': None, 'original_publish_resource_id': None, 'custom_base_model_info': None, 'training_error_details': None}
{'additional_properties': {}, 'id': 'b5903e52-fbeb-4fbf-b29f-18a804ec45b1', 'name': 'Iteration 5', 'status': 'Completed', 'created

In [39]:
model_perf = trainer.get_iteration_performance(project.id, iteration_list[5].id)

In [40]:
model_perf.as_dict()

{'per_tag_performance': [{'id': '04f2eb59-a9db-425b-be13-4e324b579698',
   'name': 'lighter',
   'precision': 1.0,
   'precision_std_deviation': 0.0,
   'recall': 0.25,
   'recall_std_deviation': 0.0,
   'average_precision': 0.9166667}],
 'precision': 1.0,
 'precision_std_deviation': 0.0,
 'recall': 0.25,
 'recall_std_deviation': 0.0,
 'average_precision': 0.9166667}

## Publishing the Model to the Project Endpoint

In [41]:
## Setting the Iteration Name, this will be used when Model training is completed
publish_iteration_name = "lighter_model"

In [42]:
# The iteration is now trained. Publish it to the project endpoint
trainer.publish_iteration(project.id, iteration.id, publish_iteration_name, prediction_resource_id)
print ("Done!")

Done!


## Performing Prediction
- Using the predictor object 

In [43]:
publish_iteration_name

'lighter_model'

In [45]:
local_image_path = '/home/workspace/lighter_test/'

In [46]:
!ls $local_image_path

lighter_test_set_1of5.jpg  lighter_test_set_3of5.jpg  lighter_test_set_5of5.jpg
lighter_test_set_2of5.jpg  lighter_test_set_4of5.jpg


In [59]:
def perform_prediction(image_file_name):
    with open(os.path.join (local_image_path,  image_file_name), "rb") as image_contents:
        results = predictor.detect_image(project.id, publish_iteration_name, image_contents.read())
        # Display the results.
        print(image_file_name)
        for prediction in results.predictions:
            if prediction.probability > 0.1:
                print("\t" + prediction.tag_name +
                      ": {0:.2f}%".format(prediction.probability * 100))
          

In [62]:
# perform_prediction('lighter_test_set_1of5.jpg')

In [61]:
for test in range(1, 6):
    image_file_name = 'lighter_test_set_{}of5.jpg'.format(test)
    perform_prediction(image_file_name)


lighter_test_set_1of5.jpg
	lighter: 70.92%
	lighter: 37.24%
	lighter: 10.72%
lighter_test_set_2of5.jpg
	lighter: 39.61%
	lighter: 38.83%
	lighter: 33.62%
	lighter: 13.17%
	lighter: 11.34%
	lighter: 10.86%
lighter_test_set_3of5.jpg
	lighter: 67.09%
	lighter: 27.05%
lighter_test_set_4of5.jpg
	lighter: 56.19%
	lighter: 22.56%
	lighter: 12.19%
lighter_test_set_5of5.jpg
	lighter: 16.26%
	lighter: 14.93%
	lighter: 10.84%
	lighter: 10.45%
